### Get DB Connection string and connect to Oracle DB

In [3]:
import import_ipynb
import Connections as conn
import cx_Oracle
#import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
#import urllib.request as urllib2
import time
#from selenium.webdriver.support.select import Select
#import re
#import os
#from io import BytesIO
#from csv import writer
import datetime

importing Jupyter notebook from Connections.ipynb


In [4]:
connstr = conn.conn_string()
connection = cx_Oracle.connect(connstr)
cursor = connection.cursor()
cursor.execute("select username from user_users")
for row in cursor.fetchall():
    print(row)
cursor.close()

('CRIC_BATCH_USER',)


### Get matchid inputs manually

In [5]:
matchid_list=[]
print('Enter the Cricinfo matchids for the required matches. To stop, please press Enter')
while True:
    text=input()
    if text:
        matchid_list.append(text)
        continue
    else:    
        break
matchid_list        

Enter the Cricinfo matchids for the required matches. To stop, please press Enter
1254058
1254059
1216493
1195618
1195611
1243389
1188428
1249239



['1254058',
 '1254059',
 '1216493',
 '1195618',
 '1195611',
 '1243389',
 '1188428',
 '1249239']

### Load matches data into Pandas DataFrame

In [6]:
row_dict = {}
for mid in matchid_list:
    driver = webdriver.Chrome("D:/Downloads/chromedriver_win32/chromedriver.exe")

    url_page_ha_scr = 'https://hsapi.espncricinfo.com/v1/pages/match/scoreboard?lang=en&leagueId=8048&eventId={match}&liveTest=false&qaTest=false'.format(match=mid)
    driver.get(url_page_ha_scr)
    content_ha_scr = driver.page_source
    page_ha_scr = BeautifulSoup(content_ha_scr)
    ha_scr=json.loads(page_ha_scr.text)

    url_page_hca_scr = 'https://hs-consumer-api.espncricinfo.com/v1/pages/match/scorecard?seriesId=366616&matchId={match}'.format(match=mid)
    driver.get(url_page_hca_scr)
    content_hca_scr = driver.page_source
    page_hca_scr = BeautifulSoup(content_hca_scr)
    hca_scr=json.loads(page_hca_scr.text)

    print('Extracting Match {} data'.format(mid))
    start_time = time.time()
    
    match_id = mid
    series_name = hca_scr['match']['series']['longName']
    match_format = ha_scr['header']['matchEvent']['generalClassName']
    season = ha_scr['header']['matchEvent']['season']
    series_match_no = hca_scr['match']['title']
    match_date = datetime.datetime.strptime(hca_scr['match']['daysInfo'], '%d %B %Y').strftime('%d-%b-%Y')
    match_time = hca_scr['match']['floodlit']
    venue = hca_scr['match']['ground']['name']
    city = hca_scr['match']['ground']['town']['name']
    country = hca_scr['match']['ground']['country']['name']
    
    home_team_name = ha_scr['header']['matchEvent']['name'].split(' v ')[0]
    away_team_name = ha_scr['header']['matchEvent']['name'].split(' v ')[1]
    
    if hca_scr['match']['teams'][0]['team']['longName'] ==  home_team_name: 
        home_team_abb = hca_scr['match']['teams'][0]['team']['abbreviation']
        away_team_abb = hca_scr['match']['teams'][1]['team']['abbreviation']        
        home_team_match_points = hca_scr['match']['teams'][0]['points']
        away_team_match_points = hca_scr['match']['teams'][1]['points']
    elif hca_scr['match']['teams'][0]['team']['longName'] ==  away_team_name: 
        home_team_abb = hca_scr['match']['teams'][1]['team']['abbreviation']
        away_team_abb = hca_scr['match']['teams'][0]['team']['abbreviation']
        home_team_match_points = hca_scr['match']['teams'][1]['points']
        away_team_match_points = hca_scr['match']['teams'][0]['points']
        
    if ha_scr['header']['matchEvent']['competitors'][0]['name'] == home_team_name: 
        home_team_captain = ha_scr['header']['matchEvent']['competitors'][0]['captain']['displayName']
        away_team_captain = ha_scr['header']['matchEvent']['competitors'][1]['captain']['displayName']
    elif ha_scr['header']['matchEvent']['competitors'][0]['name'] == away_team_name:
        home_team_captain = ha_scr['header']['matchEvent']['competitors'][1]['captain']['displayName']        
        away_team_captain = ha_scr['header']['matchEvent']['competitors'][0]['captain']['displayName']
            
    if hca_scr['content']['scorecard']['innings'][0]['inningNumber'] == 1:
        inns1_team = hca_scr['content']['scorecard']['innings'][0]['team']['longName']
        inns2_team = hca_scr['content']['scorecard']['innings'][1]['team']['longName']
        inns1_runs = hca_scr['content']['scorecard']['innings'][0]['runs']
        inns2_runs = hca_scr['content']['scorecard']['innings'][1]['runs']
        inns1_wkts = hca_scr['content']['scorecard']['innings'][0]['wickets']
        inns2_wkts = hca_scr['content']['scorecard']['innings'][1]['wickets']
        inns1_overs = hca_scr['content']['scorecard']['innings'][0]['totalOvers']
        inns2_overs = hca_scr['content']['scorecard']['innings'][1]['totalOvers']
        inns1_mins = hca_scr['content']['scorecard']['innings'][0]['minutes']
        inns2_mins = hca_scr['content']['scorecard']['innings'][1]['minutes']
        inns1_extras = hca_scr['content']['scorecard']['innings'][0]['extras']
        inns2_extras = hca_scr['content']['scorecard']['innings'][1]['extras']
        inns1_byes = hca_scr['content']['scorecard']['innings'][0]['byes']
        inns2_byes = hca_scr['content']['scorecard']['innings'][1]['byes']
        inns1_leg_byes = hca_scr['content']['scorecard']['innings'][0]['legbyes']
        inns2_leg_byes = hca_scr['content']['scorecard']['innings'][1]['legbyes']
        inns1_wides = hca_scr['content']['scorecard']['innings'][0]['wides']
        inns2_wides = hca_scr['content']['scorecard']['innings'][1]['wides']
        inns1_no_balls = hca_scr['content']['scorecard']['innings'][0]['noballs']
        inns2_no_balls = hca_scr['content']['scorecard']['innings'][1]['noballs']
        inns1_penalties = hca_scr['content']['scorecard']['innings'][0]['penalties']
        inns2_penalties = hca_scr['content']['scorecard']['innings'][1]['penalties']
        inns2_target = hca_scr['content']['scorecard']['innings'][1]['target']
    elif hca_scr['content']['scorecard']['innings'][0]['inningNumber'] == 2: 
        inns1_team = hca_scr['content']['scorecard']['innings'][1]['team']['longName']
        inns2_team = hca_scr['content']['scorecard']['innings'][0]['team']['longName']
        inns1_runs = hca_scr['content']['scorecard']['innings'][1]['runs']
        inns2_runs = hca_scr['content']['scorecard']['innings'][0]['runs']
        inns1_wkts = hca_scr['content']['scorecard']['innings'][1]['wickets']
        inns2_wkts = hca_scr['content']['scorecard']['innings'][0]['wickets']
        inns1_overs = hca_scr['content']['scorecard']['innings'][1]['totalOvers']
        inns2_overs = hca_scr['content']['scorecard']['innings'][0]['totalOvers']
        inns1_mins = hca_scr['content']['scorecard']['innings'][1]['minutes']
        inns2_mins = hca_scr['content']['scorecard']['innings'][0]['minutes']
        inns1_extras = hca_scr['content']['scorecard']['innings'][1]['extras']
        inns2_extras = hca_scr['content']['scorecard']['innings'][0]['extras']
        inns1_byes = hca_scr['content']['scorecard']['innings'][1]['byes']
        inns2_byes = hca_scr['content']['scorecard']['innings'][0]['byes']
        inns1_leg_byes = hca_scr['content']['scorecard']['innings'][1]['legbyes']
        inns2_leg_byes = hca_scr['content']['scorecard']['innings'][0]['legbyes']
        inns1_wides = hca_scr['content']['scorecard']['innings'][1]['wides']
        inns2_wides = hca_scr['content']['scorecard']['innings'][0]['wides']
        inns1_no_balls = hca_scr['content']['scorecard']['innings'][1]['noballs']
        inns2_no_balls = hca_scr['content']['scorecard']['innings'][0]['noballs']
        inns1_penalties = hca_scr['content']['scorecard']['innings'][1]['penalties']
        inns2_penalties = hca_scr['content']['scorecard']['innings'][0]['penalties']
        inns2_target = hca_scr['content']['scorecard']['innings'][0]['target']
        
    is_super_over = ha_scr['header']['isSuperOver']
    if str(ha_scr['header']['matchEvent']['competitors'][0]['isWinner'])=='True':  
        winner = ha_scr['header']['matchEvent']['competitors'][0]['name']             
    elif str(ha_scr['header']['matchEvent']['competitors'][1]['isWinner'])=='True':        
        winner = ha_scr['header']['matchEvent']['competitors'][1]['name']          
    else: 
        winner = 'Tie/NR'
    if str(ha_scr['header']['isSuperOver'])=='True': 
        result_type = ha_scr['header']['matchEvent']['statusText'].split(' (')[1].split(')')[0] 
    else: 
        result_type = ha_scr['header']['matchEvent']['statusText'].split(' won by ')[1]
    toss_winner = ha_scr['content']['about']['toss'].split(' , ')[0]
    toss_decision = ha_scr['content']['about']['toss'].split(' , ')[1]
    if 'bestPlayer' in ha_scr['header']:        
        player_of_match = ha_scr['header']['bestPlayer']['name'] 
    else: 
        player_of_match = np.nan
    if 'bestPlayer' in ha_scr['header']:        
        player_of_match_team = ha_scr['header']['bestPlayer']['teamName'] 
    else: 
        player_of_match_team = np.nan
    
    umpire1_name = hca_scr['match']['umpires'][0]['player']['longName']
    umpire1_country = hca_scr['match']['umpires'][0]['team']['longName']
    umpire1_gender = hca_scr['match']['umpires'][0]['player']['gender']
    umpire2_name = hca_scr['match']['umpires'][1]['player']['longName']
    umpire2_country = hca_scr['match']['umpires'][1]['team']['longName']
    umpire2_gender = hca_scr['match']['umpires'][1]['player']['gender']
    tv_umpire_name = hca_scr['match']['tvUmpires'][0]['player']['longName']
    tv_umpire_country = hca_scr['match']['tvUmpires'][0]['team']['longName']
    tv_umpire_gender = hca_scr['match']['tvUmpires'][0]['player']['gender']
    reserve_umpire_name = hca_scr['match']['reserveUmpires'][0]['player']['longName']
    reserve_umpire_country = hca_scr['match']['reserveUmpires'][0]['team']['longName']
    reserve_umpire_gender = hca_scr['match']['reserveUmpires'][0]['player']['gender']
    match_referee_name = hca_scr['match']['matchReferees'][0]['player']['longName']
    match_referee_country = hca_scr['match']['matchReferees'][0]['team']['longName']
    match_referee_gender = hca_scr['match']['matchReferees'][0]['player']['gender']

    col_list = [series_name,match_format,season,series_match_no,match_date,match_time,venue,city,country,home_team_name,
                home_team_abb,home_team_captain,home_team_match_points,away_team_name,away_team_abb,away_team_captain,
                away_team_match_points,inns1_team,inns1_runs,inns1_wkts,inns1_overs,inns1_mins,inns1_extras,inns1_byes,
                inns1_leg_byes,inns1_wides,inns1_no_balls,inns1_penalties,inns2_team,inns2_target,inns2_runs,inns2_wkts,inns2_overs,
                inns2_mins,inns2_extras,inns2_byes,inns2_leg_byes,inns2_wides,inns2_no_balls,inns2_penalties,is_super_over,
                winner,result_type,toss_winner,toss_decision,player_of_match,player_of_match_team,umpire1_name,umpire1_country,
                umpire1_gender,umpire2_name,umpire2_country,umpire2_gender,tv_umpire_name,tv_umpire_country,tv_umpire_gender,
                reserve_umpire_name,reserve_umpire_country,reserve_umpire_gender,match_referee_name,match_referee_country,
                match_referee_gender]
    row_dict[mid] = col_list    
    
    driver.quit()
    
    end_time = time.time()
    print('Extracted Match {} data'.format(mid))
    print('Time taken to load match data (secs):',end_time - start_time)
    print(' ')
    
df_start_time = time.time()    
matches_df = pd.DataFrame.from_dict(row_dict, orient='index', columns=
                ['series_name','match_format','season','series_match_no','match_date','match_time','venue','city',
                 'country','home_team_name','home_team_abb','home_team_captain','home_team_match_points','away_team_name',
                 'away_team_abb','away_team_captain','away_team_match_points','inns1_team','inns1_runs','inns1_wkts',
                 'inns1_overs','inns1_mins','inns1_extras','inns1_byes','inns1_leg_byes','inns1_wides','inns1_no_balls',
                 'inns1_penalties','inns2_team','inns2_target','inns2_runs','inns2_wkts','inns2_overs','inns2_mins','inns2_extras',
                 'inns2_byes','inns2_leg_byes','inns2_wides','inns2_no_balls','inns2_penalties','is_super_over','winner',
                 'result_type','toss_winner','toss_decision','player_of_match','player_of_match_team','umpire1_name',
                 'umpire1_country','umpire1_gender','umpire2_name','umpire2_country','umpire2_gender','tv_umpire_name',
                 'tv_umpire_country','tv_umpire_gender','reserve_umpire_name','reserve_umpire_country',
                 'reserve_umpire_gender','match_referee_name','match_referee_country','match_referee_gender'])
matches_df = matches_df.rename_axis('match_id').reset_index()
matches_df['match_id'] = matches_df['match_id'].astype(str).astype(int)
df_end_time = time.time()

print(' ')
print('Time taken to create matches df (secs):',df_end_time - df_start_time)
print('Matches data Extraction over')

Extracting Match 1254058 data
Extracted Match 1254058 data
Time taken to load match data (secs): 3.6266233921051025
 
Extracting Match 1254059 data
Extracted Match 1254059 data
Time taken to load match data (secs): 2.554271936416626
 
Extracting Match 1216493 data
Extracted Match 1216493 data
Time taken to load match data (secs): 2.2417492866516113
 
Extracting Match 1195618 data
Extracted Match 1195618 data
Time taken to load match data (secs): 2.660202980041504
 
Extracting Match 1195611 data


IndexError: list index out of range

### Insert Pandas DataFrame into Oracle table

In [ ]:
print('Data insertion into Oracle table begins')
cursor = connection.cursor()
rows = [tuple(x) for x in matches_df.values]
sql_qry = ("INSERT INTO tgt_t20_dbo.matches (match_id,series_name,match_format,season,series_match_no,match_date,match_time,"
           "venue,city,country,home_team_name,home_team_abb,home_team_captain,home_team_match_points,away_team_name,"
           "away_team_abb,away_team_captain,away_team_match_points,inns1_team,inns1_runs,inns1_wkts,inns1_overs,inns1_mins,"
           "inns1_extras,inns1_byes,inns1_leg_byes,inns1_wides,inns1_no_balls,inns1_penalties,inns2_team,inns2_target,inns2_runs,inns2_wkts,"
           "inns2_overs,inns2_mins,inns2_extras,inns2_byes,inns2_leg_byes,inns2_wides,inns2_no_balls,inns2_penalties,"
           "is_super_over,winner,result_type,toss_winner,toss_decision,player_of_match,player_of_match_team,umpire1_name,"
           "umpire1_country,umpire1_gender,umpire2_name,umpire2_country,umpire2_gender,tv_umpire_name,tv_umpire_country,"
           "tv_umpire_gender,reserve_umpire_name,reserve_umpire_country,reserve_umpire_gender,match_referee_name,"
           "match_referee_country,match_referee_gender)" 
           "VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23,:24,:25,:26,:27,:28,"
           ":29,:30,:31,:32,:33,:34,:35,:36,:37,:38,:39,:40,:41,:42,:43,:44,:45,:46,:47,:48,:49,:50,:51,:52,:53,:54,:55,:56,"
           ":57,:58,:59,:60,:61,:62)")
cursor.executemany(sql_qry,rows)
connection.commit()
cursor.close()
connection.close()
print('Data insertion ends')